In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

!pip install ultralytics

In [2]:
from ultralytics import YOLO
import json
import glob
import os
from pathlib import Path

In [3]:
directory_name = "gdrive/MyDrive/Colab Notebooks/GESTALT/"
json_filename = "gdrive/MyDrive/Colab Notebooks/GESTALT/metadata"

In [8]:
def detect_tags_from_jpgs_in_directory(directory_name, json_filename):
    # Find all jpegs in the directory
    full_list_of_img_files = glob.glob(os.path.join(directory_name,"*.jpg"))

    # Chunk images into lists of size MAX_FILES_OPEN
    MAX_FILES_OPEN = 200
    for i in range(0, len(full_list_of_img_files), MAX_FILES_OPEN):
        list_of_img_files = full_list_of_img_files[i : i + MAX_FILES_OPEN]

        # Read in json file
        json_dict = json.load(open(json_filename+".json"))

        # Instantiate model
        model = YOLO("yolov8m.pt")

        # Run images through model and get results
        results = model.predict(list_of_img_files, verbose=False)

        # Check that we got results for every image
        assert len(results) == len(list_of_img_files)

        # Loop over each image
        for idx in range(len(list_of_img_files)):
            file_id = Path(list_of_img_files[idx]).stem  # extract filename without path or extension
            result = results[idx]
            tags = []
            coords = []
            probs = []
            # Loop over each object detected in the image
            for box in result.boxes:
                class_id = result.names[box.cls[0].item()]
                cords = box.xyxy[0].tolist()
                cords = [round(x) for x in cords]
                conf = round(box.conf[0].item(), 2)

                tags.append(class_id)  # object type like person or bench
                coords.append(cords)  # coordinates in the image like [121, 632, 207, 732]
                probs.append(conf)  # confidence score like 0.81

            try:
                json_dict[file_id].update({"objects":tags, "coordinates":coords, "probabilities":probs})
            except KeyError:
                print("NOT FOUND:",file_id,"\n Tags:",tags,"\n coords:", coords, "\n probs", probs)

    return json.dumps(json_dict,indent=4)

In [9]:
output = detect_tags_from_jpgs_in_directory(directory_name, json_filename)

In [10]:
output

'{\n    "52896798685": {\n        "photo_bearing_ref": "NULL",\n        "photo_bearing": "NULL",\n        "photo_date": "2023:05:03 12:48:52",\n        "longitude": "116.002501",\n        "latitude": "-31.889322",\n        "URL": "https://live.staticflickr.com/65535/52896798685_b1d0f647e2_b.jpg"\n    },\n    "52874952494": {\n        "photo_bearing_ref": "True North",\n        "photo_bearing": "81.11177052",\n        "photo_date": "2023:04:11 14:20:29",\n        "longitude": "115.987952",\n        "latitude": "-31.854648",\n        "URL": "https://live.staticflickr.com/65535/52874952494_d070a7eb16_b.jpg"\n    },\n    "52874206492": {\n        "photo_bearing_ref": "True North",\n        "photo_bearing": "171.9923018",\n        "photo_date": "2023:04:11 10:31:22",\n        "longitude": "115.963608",\n        "latitude": "-31.875187",\n        "URL": "https://live.staticflickr.com/65535/52874206492_fa8a4af132_b.jpg"\n    },\n    "52874785511": {\n        "photo_bearing_ref": "True North",

In [11]:
with open(json_filename+"_objects.json", "w") as out:
    out.write(output)